In [ ]:
import pandas as pd
import plotly.express as px
import xarray as xr

pd.options.plotting.backend = "plotly"

## Load dataset

---


In [2]:
bats_bottle = xr.load_dataset("../../2_processed/bats_bottle.nc")
bats_primary_production = xr.load_dataset(
    "../../2_processed/bats_primary_production.nc"
)
bats_zooplankton = xr.load_dataset("../../2_processed/bats_zooplankton.nc")

In [3]:
bats_zooplankton = bats_zooplankton[
    ["dry_weight_vol_water_ratio", "wet_weight_vol_water_ratio"]
]
bats_zooplankton

<xarray.Dataset> Size: 610kB
Dimensions:                     (time: 541, is_day: 2, depth: 7, latitude: 1,
                                 longitude: 1, sieve_size: 5)
Coordinates:
  * time                        (time) datetime64[ns] 4kB 1995-05-10 ... 2022...
  * is_day                      (is_day) bool 2B False True
  * depth                       (depth) float64 56B 50.0 100.0 ... 300.0 350.0
  * latitude                    (latitude) float64 8B 31.6
  * longitude                   (longitude) float64 8B -64.2
  * sieve_size                  (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
Data variables:
    dry_weight_vol_water_ratio  (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...
    wet_weight_vol_water_ratio  (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB ...

In [4]:
bats_primary_production

<xarray.Dataset> Size: 260kB
Dimensions:    (time: 445, depth: 8, latitude: 1, longitude: 1)
Coordinates:
  * time       (time) datetime64[ns] 4kB 1988-12-18 ... 2022-12-16T08:11:00
  * depth      (depth) int64 64B 50 100 150 200 250 300 350 400
  * latitude   (latitude) float64 8B 31.6
  * longitude  (longitude) float64 8B -64.2
Data variables:
    pres       (time, depth, latitude, longitude) float64 28kB nan nan ... nan
    temp       (time, depth, latitude, longitude) float64 28kB nan nan ... nan
    salt       (time, depth, latitude, longitude) float64 28kB nan nan ... nan
    lt1        (time, depth, latitude, longitude) float64 28kB 6.605 ... nan
    lt2        (time, depth, latitude, longitude) float64 28kB 6.59 ... nan
    lt3        (time, depth, latitude, longitude) float64 28kB nan 2.37 ... nan
    dark       (time, depth, latitude, longitude) float64 28kB 0.75 ... nan
    t0         (time, depth, latitude, longitude) float64 28kB 1.615 ... nan
    pp         (time, depth, latitude, longitude) float64 28kB 6.15 1.22 ... nan

In [5]:
bats_bottle

<xarray.Dataset> Size: 7MB
Dimensions:                        (time: 3754, depth: 8, latitude: 1,
                                    longitude: 1)
Coordinates:
  * time                           (time) datetime64[ns] 30kB 1988-10-21T04:5...
  * depth                          (depth) int64 64B 50 100 150 ... 300 350 400
  * latitude                       (latitude) float64 8B 31.6
  * longitude                      (longitude) float64 8B -64.2
Data variables: (12/27)
    temp                           (time, depth, latitude, longitude) float64 240kB ...
    CTD_salinity                   (time, depth, latitude, longitude) float64 240kB ...
    salinity                       (time, depth, latitude, longitude) float64 240kB ...
    sigma_theta                    (time, depth, latitude, longitude) float64 240kB ...
    oxygen                         (time, depth, latitude, longitude) float64 240kB ...
    oxygen_fix_temp                (time, depth, latitude, longitude) float64 240kB ...
    ...                             ...
    particulate_biogenic_silica    (time, depth, latitude, longitude) float64 240kB ...
    particulate_lithogenic_silica  (time, depth, latitude, longitude) float64 240kB ...
    prochlorococcus                (time, depth, latitude, longitude) float64 240kB ...
    synechococcus                  (time, depth, latitude, longitude) float64 240kB ...
    picoeukaryotes                 (time, depth, latitude, longitude) float64 240kB ...
    nanoeukaryotes                 (time, depth, latitude, longitude) float64 240kB ...

## Manage index

---


In [6]:
times = pd.DataFrame(
    {
        "btl_time": bats_bottle.time.to_series(),
        "zpk_time": bats_zooplankton.time.to_series(),
        "pp_time": bats_primary_production.time.to_series(),
    }
)
fig = px.histogram(
    times,
    x=["btl_time", "zpk_time", "pp_time"],
    title="time distribution",
    opacity=0.5,
    log_y=True,
    nbins=30,
    barmode="overlay",
    marginal="box",
    labels={"value": "time (m)", "variable": "Datasets"},
)

fig.show()

In [7]:
depths = pd.DataFrame(
    {
        "ctd_depth": bats_bottle.depth.to_series(),
        "zpk_depth": bats_zooplankton.depth.to_series(),
        "pp_depth": bats_primary_production.depth.to_series(),
    }
)
px.box(
    depths.stack().reset_index().rename(columns={"level_1": "dataset"}),
    x="dataset",
    y="depth",
)

## Migrant and resident


## Gathering

---


# TODO : Select only the wanted variables in each dataset


In [8]:
# Temporary, need to select the temp to keep
bats_bottle = bats_bottle.drop_vars("temp")

In [9]:
final_dataset = xr.merge([bats_zooplankton, bats_primary_production, bats_bottle])
final_dataset.load()

<xarray.Dataset> Size: 16MB
Dimensions:                        (time: 4428, is_day: 2, depth: 8,
                                    latitude: 1, longitude: 1, sieve_size: 5)
Coordinates:
  * time                           (time) datetime64[ns] 35kB 1988-10-21T04:5...
  * is_day                         (is_day) bool 2B False True
  * depth                          (depth) float64 64B 50.0 100.0 ... 400.0
  * latitude                       (latitude) float64 8B 31.6
  * longitude                      (longitude) float64 8B -64.2
  * sieve_size                     (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
Data variables: (12/37)
    dry_weight_vol_water_ratio     (time, is_day, depth, latitude, longitude, sieve_size) float64 3MB ...
    wet_weight_vol_water_ratio     (time, is_day, depth, latitude, longitude, sieve_size) float64 3MB ...
    pres                           (time, depth, latitude, longitude) float64 283kB ...
    temp                           (time, depth, latitude, longitude) float64 283kB ...
    salt                           (time, depth, latitude, longitude) float64 283kB ...
    lt1                            (time, depth, latitude, longitude) float64 283kB ...
    ...                             ...
    particulate_biogenic_silica    (time, depth, latitude, longitude) float64 283kB ...
    particulate_lithogenic_silica  (time, depth, latitude, longitude) float64 283kB ...
    prochlorococcus                (time, depth, latitude, longitude) float64 283kB ...
    synechococcus                  (time, depth, latitude, longitude) float64 283kB ...
    picoeukaryotes                 (time, depth, latitude, longitude) float64 283kB ...
    nanoeukaryotes                 (time, depth, latitude, longitude) float64 283kB ...

In [10]:
final_dataset.to_netcdf("../../3_post_processed/bats_product.nc", mode="w")

/var/folders/z_/8j3qx1mn0299kkpjgz9g53780000gq/T/ipykernel_5753/849147891.py:1: UserWarning:

Times can't be serialized faithfully to int64 with requested units 'days since 1995-05-10'. Serializing with units 'minutes since 1995-05-10' instead. Set encoding['dtype'] to floating point dtype to serialize with units 'days since 1995-05-10'. Set encoding['units'] to 'minutes since 1995-05-10' to silence this warning .

